# A preliminary approach to corpus driven research of Persian classical music
Babak Nikzat and Rafael Caro Repetto

This is the code that ...

In [1]:
import essentia
import essentia.standard as es
import essentiaUtils as eu
import matplotlib.pyplot as plt
import numpy as np
import IPython
import intonation
import os

In [ ]:
# kdc_folder = '../KDC'

# kdc_recordings = []

# kdc_cr = os.listdir(os.path.join(kdc_folder, 'KDC-CR'))
# for r in kdc_cr:
#     if r[-4:] == 'flac':
#         kdc_recordings.append(os.path.join(kdc_folder, r))
        
# kdc_or = os.listdir(os.path.join(kdc_folder, 'KDC-OR'))
# for r in kdc_or:
#     if r[-4:] == 'flac':
#         kdc_recordings.append(os.path.join(kdc_folder, r))
        
# for i in range(len(kdc_recordings)):
#     print('{} : {}'.format(i, kdc_recordings[i].split('/')[-1]))

In [2]:
kdc_folder = '../KDC'

data_path = os.path.join(kdc_folder, 'KDC-data.csv')

with open(data_path, 'r') as f:
    kdc_data = f.readlines()

In [4]:
search_items = ['Abuata', 'Afshari', 'Bayat-e Tork', 'Chahargah', 'Dashti', 'Esfahan', 'Homayun', 'Nava', 'Rast-Panjgah', 'Segah', 'Shur']
search_column = 'Dastgah'

search_column_index = None
heading = kdc_data[0].rstrip().split(';')
for i in range(len(heading)):
    if search_column == heading[i]:
        search_column_index = i
if search_column_index == None:
    print('ERROR: no search column has been found')

recordings = {}
    
for row in kdc_data[1:]:
    row_data = row.rstrip().split(';')
    if row_data[search_column_index] in search_items:
        recording_path = os.path.join(kdc_folder, 'KDC-'+row_data[1], row_data[0])
        shahed = row_data[7]
        minf0 = row_data[8]
        maxf0 = row_data[9]
        f0_cf = row_data[10]
        if shahed == '':
            shahed = None
        if minf0 == '':
            minf0 = '20'
        if maxf0 == '':
            maxf0 = '22050'
        if f0_cf == '':
            f0_cf = '0.9'
        recordings[recording_path] = {'shahed': shahed, 'minf0':float(minf0), 'maxf0':float(maxf0), 'f0_cf':float(f0_cf)}

print('{} recordings found:'.format(len(recordings)))
recordingsPaths = list(recordings.keys())
for i in range(len(recordingsPaths)):
    print('    {}: {}'.format(i, recordingsPaths[i].split('/')[-1]))

27 recordings found:
    0: ABD_N_MKH_B.flac
    1: ABD_N_MKH_E.flac
    2: AFD_N_MKH_B.flac
    3: AFD_N_MKH_E.flac
    4: BAD_N_MKH_B.flac
    5: BAD_N_MKH_E.flac
    6: CHD1_N_AA_E.flac
    7: CHD1_T_MH_E.flac
    8: CHD2_N_AA_E.flac
    9: CHD2_T_MH_E.flac
    10: CHD3_T_MH_E.flac
    11: CHD_N_MKH_B.flac
    12: CHD_N_MKH_E.flac
    13: DAD_N_MKH_B.flac
    14: DAD_N_MKH_E.flac
    15: ESD_N_MKH_B.flac
    16: ESD_N_MKH_E.flac
    17: HOD_N_MKH_B.flac
    18: HOD_N_MKH_E.flac
    19: NAD_N_MKH_B.flac
    20: NAD_N_MKH_E.flac
    21: RAD_N_MKH_B.flac
    22: RAD_N_MKH_E.flac
    23: SED_N_MKH_B.flac
    24: SED_N_MKH_E.flac
    25: SHD_N_MKH_B.flac
    26: SHD_N_MKH_E.flac


## Analysis of single track

### Compute pitch track

#### Load audio

In [ ]:
recording_index = 5

recordingPath = recordingsPaths[recording_index]
recording = recordings[recordingPath]

print(recordingPath.split('/')[-1])

IPython.display.Audio(recordingPath)

In [ ]:
loader = es.MonoLoader(filename=recordingPath)
eqLoud = es.EqualLoudness()

audio = eqLoud(loader())

#### Extract pitch track

In [ ]:
windowSize = 2048
hopSize = 128

In [ ]:
f0 = eu.get_f0(audio, minf0=recording['minf0'], maxf0=recording['maxf0'], cf=recording['f0_cf'], ws=windowSize, hs=hopSize)
timeStamps = np.arange(f0.size)*hopSize/44100

#### Plot with spectrogram

In [ ]:
x, y, z = eu.spectrogram(audio, ws=windowSize, hs=hopSize)

In [ ]:
lim_x = (0, 10)    # Needs a tuple
lim_y = (800, 1500)    # Needs a tuple

plt.rcParams['figure.figsize'] = (15, 6)

plt.pcolormesh(x, y, z)
plt.plot(timeStamps[f0>0], f0[f0>0], '.k', markersize=0.5)
plt.xlim(lim_x)
plt.ylim(lim_y)
plt.show()

### Compute histogram (Hz)

In [ ]:
pitch_obj = intonation.Pitch(timeStamps[f0>20], f0[f0>0])

In [ ]:
rec_obj = intonation.Recording(pitch_obj)

In [ ]:
rec_obj.compute_hist(bins=1000)
rec_obj.histogram.get_peaks()
rec_obj.histogram.plot()

In [ ]:
print(recordingPath.split('/')[-1])
print()

peaks = rec_obj.histogram.peaks['peaks']
for i in range(len(peaks[0])):
    print('{}. {} : {:.6f}'.format(i, round(peaks[0][i]), peaks[1][i]))

### Compute histogram (cents)

In [ ]:
shahed = recording['shahed']
one_octave = False

if shahed == None:
    print('ERROR: There is no shahed computed for this recording')
else:
    print('Shahed for {}: {}'.format(recordingPath.split('/')[-1], shahed))

In [ ]:
f0cents = 1200 * np.log2(f0[f0>0] / float(shahed))

In [ ]:
pitch_cents_obj = intonation.Pitch(timeStamps[f0>0], f0cents)

In [ ]:
rec_cents_obj = intonation.Recording(pitch_cents_obj)

In [ ]:
rec_cents_obj.compute_hist(bins=1200, folded=one_octave)
rec_cents_obj.histogram.get_peaks()
rec_cents_obj.histogram.plot()

In [ ]:
print(recordingPath.split('/')[-1])
print()

if one_octave:
    print('   0 : {:.6f}'.format(rec_cents_obj.histogram.y[0]))

peaks_cents = rec_cents_obj.histogram.peaks['peaks']
for i in range(len(peaks_cents[0])):
    print('{}. {} : {:.6f}'.format(i, round(peaks_cents[0][i]), peaks_cents[1][i]))

### Vibrato analysis

In [ ]:
vib = es.Vibrato(sampleRate = 44100/hopSize)
vibF, vibE = vib(essentia.array(f0))

In [ ]:
lim_x = None    # Needs a tuple
lim_y = None    # Needs a tuple

plt.rcParams['figure.figsize'] = (15, 6)

plt.figure()
plt.subplot(311)
plt.plot(timeStamps[f0>0], f0[f0>0], '.k', markersize=0.5)
plt.xlim(lim_x)
plt.ylim(lim_y)
plt.title('f0')
plt.subplot(312)
plt.plot(timeStamps, vibF)
plt.xlim(lim_x)
plt.title('frequency')
plt.subplot(313)
plt.plot(timeStamps, vibE)
plt.xlim(lim_x)
plt.title('extent')
plt.tight_layout()
plt.show()

In [ ]:
print(recordingPath.split('/')[-1])
print()

freq_mean = np.mean(vibF[vibF>0])
freq_sd = np.std(vibF[vibF>0])
print('Frequency:\tmean: {:.2f} Hz\t\tSD: {:.2f} Hz'.format(freq_mean, freq_sd))

ex_mean = np.mean(vibE[vibE>20])
ex_sd = np.std(vibE[vibE>20])
print('Extent:\t\tmean: {:.2f} cents\tSD: {:.2f} cents'.format(ex_mean, ex_sd))

### Loudness analysis

In [ ]:
loud = es.Loudness()
energy = es.Energy()
rms = es.RMS()

loudTrack = []
energyTrack = []
rmsTrack = []

for frame in es.FrameGenerator(audio, frameSize=2048, hopSize=hopSize):
    frameLoud = loud(frame)
    frameEnergy = energy(frame)
    frameRms = rms(frame)
    loudTrack.append(frameLoud)
    energyTrack.append(frameEnergy)
    rmsTrack.append(frameRms)

In [ ]:
lim_x = None       # Needs a tuple
lim_y = None         # Needs a tuple
lim_y_loud = None    # Needs a tuple

plt.rcParams['figure.figsize'] = (15, 6)

plt.figure()
plt.subplot(411)
plt.plot(timeStamps, f0, '.k', markersize=0.5)
plt.xlim(lim_x)
plt.ylim(lim_y)
plt.title('pitch')
plt.subplot(412)
plt.plot(timeStamps, loudTrack)
plt.xlim(lim_x)
plt.ylim(lim_y_loud)
plt.title('loudness (Loudness)')
plt.subplot(413)
plt.plot(timeStamps, energyTrack)
plt.xlim(lim_x)
plt.ylim(lim_y_loud)
plt.title('loudness (Energy)')
plt.subplot(414)
plt.plot(timeStamps, rmsTrack)
plt.xlim(lim_x)
plt.ylim(lim_y_loud)
plt.title('loudness (RMS)')
plt.tight_layout()
plt.show()

### Start and end analysis

In [ ]:
pitchContourSegmentation = es.PitchContourSegmentation()

midiNotes = pitchContourSegmentation(essentia.array(f0), audio)

In [ ]:
def midi2Hz(midi):
    '''
    Converts a given midi value to its equivalent in hertz
    
    Args:
        midi (str): midi value as string
        
    Returns:
        frequency (str): equivalent frequency in Hertz as string
        
    >>> midi2Hz('60')
    '261.6255653005986'
    '''
    f = 2 ** ((float(midi) - 69) / 12.) * 440
    return str(f)

In [ ]:
# txt = ''
# for i in range(len(midiNotes[0])):
#     txt += str(midiNotes[0][i]) + '\t' + str(midiNotes[1][i]) + '\t' + midi2Hz(midiNotes[2][i]) + '\n'

# with open(recordingPath[:-5]+'-notes.txt', 'w') as f:
#     f.write(txt.rstrip())

In [ ]:
for i in range(10):
    print('{:.2f}: {:.2f}'.format(midiNotes[1][i], midiNotes[2][i]))

In [ ]:
first_start = midiNotes[0][0]
first_end = first_start + midiNotes[1][0]

i = 0
j = 0
dur = midiNotes[1][0]
while dur < 1:
    i += 1
    j = 0
    dur = midiNotes[1][i]
    midi = midiNotes[2][i]
    while midiNotes[2][i+j+1] == midi:
        dur += midiNotes[1][i+j+1]
        j += 1
first_long_start = midiNotes[0][i]
first_long_end = first_long_start + dur

last_start = midiNotes[0][-1]
last_end = last_start + midiNotes[1][-1]

first_start_i = np.abs(timeStamps - first_start).argmin()
first_end_i = np.abs(timeStamps - first_end).argmin()

first_long_start_i = np.abs(timeStamps - first_long_start).argmin()
first_long_end_i = np.abs(timeStamps - first_long_end).argmin()

last_start_i = np.abs(timeStamps - last_start).argmin()
last_end_i = np.abs(timeStamps - last_end).argmin()

first = f0[first_start_i:first_end_i+1]
first_long = f0[first_long_start_i:first_long_end_i+1]
last = f0[last_start_i:last_end_i+1]

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)

plt.figure()
plt.subplot(311)
plt.plot(timeStamps[first_start_i:first_end_i+1], first)
plt.h
plt.title('first')
plt.subplot(312)
plt.plot(timeStamps[first_long_start_i:first_long_end_i+1], first_long)
plt.title('first long')
plt.subplot(313)
plt.plot(timeStamps[last_start_i:last_end_i+1], last)
plt.title('last')
plt.tight_layout()
plt.show()

In [ ]:
print(recordingPath.split('/')[-1])
print('Shahed: {}'.format(recording['shahed']))
print('First note (mean): {}'.format(round(np.mean(first))))
print('First long note (mean): {}'.format(round(np.mean(first_long))))
print('Last note (mean): {}'.format(round(np.mean(last))))

## Analysis of multiple recordings

### Select recordings

In [ ]:
selected_indexes = []    # If none given, all recordings selected

selected_recordings = {}

if len(selected_indexes) == 0:
    selected_indexes = range(len(recordings.keys()))
    
for i in range(len(selected_indexes)):
    recordingPath = recordingsPaths[i]
    recording = recordings[recordingPath]
    if recording['shahed'] == None:
        print('The recording {} has no shahed: it is not selected'.format(recordingPath.split('/')[-1]))
    else:
        selected_recordings[recordingPath] = recording

print()
print('{} recordings selected:'.format(len(selected_recordings)))
for k in selected_recordings.keys():
    print('    {}'.format(k.split('/')[-1]))

### Aggregated pitch track (cents)

In [ ]:
windowSize = 2048
hopSize = 128

eqLoud = es.EqualLoudness()

agg_f0_cents = np.array([])

for k in selected_recordings.keys():
    recording = selected_recordings[k]
    loader = es.MonoLoader(filename=k)
    print('Loading {}'.format(k.split('/')[-1]))
    audio = eqLoud(loader())
    f0 = eu.get_f0(audio, minf0=recording['minf0'], maxf0=recording['maxf0'], cf=recording['f0_cf'], ws=windowSize, hs=hopSize)
    f0cents = 1200 * np.log2(f0[f0>0] / float(recording['shahed']))
    agg_f0_cents = np.append(agg_f0_cents, f0cents)
print('Done!')

### Aggregated pitch histogram (cents)

In [ ]:
one_octave=True

agg_pitch_cents_obj = intonation.Pitch(np.arange(len(agg_f0_cents[agg_f0_cents>0])), agg_f0_cents)

In [ ]:
agg_rec_cents_obj = intonation.Recording(agg_pitch_cents_obj)

In [ ]:
agg_rec_cents_obj.compute_hist(bins=1200, folded=one_octave)
agg_rec_cents_obj.histogram.get_peaks()
agg_rec_cents_obj.histogram.plot()

In [ ]:
if one_octave:
    print('   0 : {:.6f}'.format(agg_rec_cents_obj.histogram.y[0]))

agg_peaks_cents = agg_rec_cents_obj.histogram.peaks['peaks']
for i in range(len(agg_peaks_cents[0])):
    print('{}. {} : {:.6f}'.format(i, round(agg_peaks_cents[0][i]), agg_peaks_cents[1][i]))

### Start and end analysis

In [ ]:
def midi2Hz(midi):
    '''
    Converts a given midi value to its equivalent in hertz
    
    Args:
        midi (str): midi value as string
        
    Returns:
        frequency (str): equivalent frequency in Hertz as string
        
    >>> midi2Hz('60')
    '261.6255653005986'
    '''
    f = 2 ** ((float(midi) - 69) / 12.) * 440
    return str(f)

In [ ]:
windowSize = 2048
hopSize = 128

eqLoud = es.EqualLoudness()
pitchContourSegmentation = es.PitchContourSegmentation()

message = 'Recording\t\tShahed\t\tFirst note\tFirst long note\t\tLast note\n'

for  k in selected_recordings.keys():
    message += k.split('/')[-1]
    recording = selected_recordings[k]
    if len(k.split('/')[-1]) > 15:
        sep = '\t '
    else:
        sep = '\t\t '
    message += sep + selected_recordings[k]['shahed'] + '\t\t '
    loader = es.MonoLoader(filename=k)
    print('Loading {}'.format(k.split('/')[-1]))
    audio = eqLoud(loader())

    print('  Computing pitch track...')
    f0 = eu.get_f0(audio, minf0=recording['minf0'], maxf0=recording['maxf0'], cf=recording['f0_cf'], ws=windowSize, hs=hopSize)

    print('  Computing notes...')
    midiNotes = pitchContourSegmentation(essentia.array(f0), audio)

    first_start = midiNotes[0][0]
    first_end = first_start + midiNotes[1][0]
    i = 0
    j = 0
    dur = midiNotes[1][0]
    while dur < 1:
        i += 1
        j = 0
        dur = midiNotes[1][i]
        midi = midiNotes[2][i]
        while midiNotes[2][i+j+1] == midi:
            dur += midiNotes[1][i+j+1]
            j += 1
    first_long_start = midiNotes[0][i]
    first_long_end = first_long_start + dur

    last_start = midiNotes[0][-1]
    last_end = last_start + midiNotes[1][-1]

    first_start_i = np.abs(timeStamps - first_start).argmin()
    first_end_i = np.abs(timeStamps - first_end).argmin()

    first_long_start_i = np.abs(timeStamps - first_long_start).argmin()
    first_long_end_i = np.abs(timeStamps - first_long_end).argmin()

    last_start_i = np.abs(timeStamps - last_start).argmin()
    last_end_i = np.abs(timeStamps - last_end).argmin()

    first = f0[first_start_i:first_end_i+1]
    first_long = f0[first_long_start_i:first_long_end_i+1]
    last = f0[last_start_i:last_end_i+1]
    
    message += str(round(np.mean(first))) + '\t\t ' + str(round(np.mean(first_long))) + '\t\t\t ' + str(round(np.mean(last))) + '\n'

print()
print(message)

## Preprocessing steps

### Find pitch range

#### Load audio

In [55]:
recording_index = 2

recordingPath = recordingsPaths[recording_index]
recording = recordings[recordingPath]

print(recordingPath.split('/')[-1])

IPython.display.Audio(recordingPath)

AFD_N_MKH_B.flac


In [56]:
loader = es.MonoLoader(filename=recordingPath)
eqLoud = es.EqualLoudness()

audio = eqLoud(loader())

#### Extract pitch track

In [57]:
windowSize = 2048
hopSize = 128

In [58]:
for k in recording:
    print(k + ': ' + str(recording[k]))

shahed: None
minf0: 20.0
maxf0: 22050.0
f0_cf: 0.9


In [71]:
f0 = eu.get_f0(audio, minf0=290, maxf0=500, cf=0.6, ws=windowSize, hs=hopSize)

In [72]:
print(np.floor(min(f0[f0>0])))
print(np.ceil(max(f0)))

293.0
486.0


In [73]:
txt = ''
for i in f0:
    txt += str(i) + '\n'

with open(recordingPath[:-5]+'-f0.txt', 'w') as f:
    f.write(txt.rstrip())

#### Plot with spectrogram

In [ ]:
x, y, z = eu.spectrogram(audio, ws=windowSize, hs=hopSize)

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)

plt.pcolormesh(x, y, z)
plt.plot(x[f0>20], f0[f0>20], '.k', markersize=0.5)
plt.ylim([0, 1500])
plt.show()

In [ ]:
eu.get_f0?